In [1]:
from collections import defaultdict

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from data import tfrecord_labeled_dataset, tfrecord_dataset, train_test_split, kfolds
from layers import SeluConv3D, SeluDense
from plot import plot_volume_animation, plot_loss_history
from config import (
    CT_0_TFRECORD,
    CT_1_TFRECORD,
    CT_2_TFRECORD,
    CT_3_TFRECORD,
    CT_4_TFRECORD,
    COVID_NEG_TFRECORD,
    COVID_POS_TFRECORD,
    SCAN_SHAPE,
    LIDC_NUM_NODULES_TFRECORD,
)

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 7]

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# LIDC dataset

In [2]:
lidc_dataset = tfrecord_labeled_dataset(LIDC_NUM_NODULES_TFRECORD)
# lidc_samples = sum(1 for _ in tqdm(lidc_dataset))
lidc_samples = 1141
print(f"{lidc_samples = }")
lidc_dataset

lidc_samples = 1141


<ParallelMapDataset shapes: ((None, None, None, None), (1,)), types: (tf.float32, tf.uint16)>

In [ ]:
def build_regression_model():
    model = keras.Sequential(
        [
            keras.layers.InputLayer(SCAN_SHAPE, name="input_layer"),
            SeluConv3D(filters=32, kernel_size=3, name="selu_conv3d_1"),
            keras.layers.MaxPool3D(2, name="maxpool3d_1"),
            SeluConv3D(filters=64, kernel_size=3, name="selu_conv3d_2"),
            keras.layers.MaxPool3D(2, name="maxpool3d_2"),
            SeluConv3D(filters=128, kernel_size=3, name="selu_conv3d_3"),
            keras.layers.MaxPool3D(2, name="maxpool3d_3"),
            SeluConv3D(filters=256, kernel_size=3, name="selu_conv3d_4"),
            keras.layers.MaxPool3D(2, name="maxpool3d_4"),
            keras.layers.Flatten(name="flatten"),
            keras.layers.Dense(1, name="final_dense"),
        ],
        name="3d_cnn",
    )
    return model


m = build_regression_model()
m.summary()

In [ ]:
val_perc = 0.2
learning_rate = 1e-4
batch_size = 16
patience = 15

In [ ]:
train_dataset, val_dataset = train_test_split(
    lidc_dataset, test_perc=val_perc, cardinality=lidc_samples
)
val_dataset = (
    val_dataset.cache().batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
)
train_dataset = (
    train_dataset.cache()  # must be called before shuffle
    .shuffle(buffer_size=1024, reshuffle_each_iteration=True)
    .batch(batch_size)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [ ]:
cnn = build_regression_model()
cnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=keras.losses.MeanSquaredError(),
)
history = cnn.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1000,
    verbose=1,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=patience,
            restore_best_weights=True,
        ),
    ],
)
cnn.save("models/lidc-num-nodules-3dcnn.h5")

In [ ]:
plot_loss_history(history)
plt.savefig("num-nodules-regression-loss.pdf")

In [ ]:
# cnn = keras.models.load_model("models/")
for x, y in val_dataset.unbatch().batch(1).take(10).as_numpy_iterator():
    print(f"real: {y}")
    print(f"predicted: {cnn(x, training=False).numpy()}")
    print()

# Covid dataset

In [3]:
neg_x = tfrecord_dataset([CT_0_TFRECORD])
# neg_samples = sum(1 for _ in tqdm(neg_x))
neg_samples = 254  # CT-0
print(f"{neg_samples = }")
neg_dataset = tf.data.Dataset.zip(
    (neg_x, tf.data.Dataset.from_tensor_slices(np.int8([[0]])).repeat(neg_samples))
)
# assert sum(1 for _ in tqdm(neg_dataset)) == neg_samples

pos_x = tfrecord_dataset([CT_1_TFRECORD, CT_2_TFRECORD, CT_3_TFRECORD, CT_4_TFRECORD])
# pos_samples = sum(1 for _ in tqdm(pos_x))
pos_samples = 856  # CT-1 + CT-2 + CT-3 + CT-4
# pos_samples = 127  # CT-2 + CT-3 + CT-4
# pos_x = pos_x.shard(3, index=0)
# pos_samples = sum(1 for _ in pos_x)
print(f"{pos_samples = }")
pos_dataset = tf.data.Dataset.zip(
    (pos_x, tf.data.Dataset.from_tensor_slices(np.int8([[1]])).repeat(pos_samples))
)
# assert sum(1 for _ in tqdm(pos_dataset)) == pos_samples

covid_dataset = neg_dataset.concatenate(pos_dataset)
covid_samples = neg_samples + pos_samples
# assert sum(1 for _ in tqdm(covid_dataset)) == covid_samples
print(f"{covid_samples = }")
covid_dataset

neg_samples = 254
pos_samples = 856
covid_samples = 1110


<ConcatenateDataset shapes: ((None, None, None, None), (1,)), types: (tf.float32, tf.int8)>

In [4]:
# scan, label = next(iter(covid_dataset.skip(300)))
# print(f"{label = }")
# plot_volume_animation(scan)

label = <tf.Tensor: shape=(1,), dtype=int8, numpy=array([1], dtype=int8)>


In [ ]:
def build_3d_cnn():
    model = keras.Sequential(
        [
            keras.layers.InputLayer(SCAN_SHAPE, name="input_layer"),
            SeluConv3D(filters=32, kernel_size=3, name="selu_conv3d_1"),
            keras.layers.MaxPool3D(2, name="maxpool3d_1"),
            SeluConv3D(filters=64, kernel_size=3, name="selu_conv3d_2"),
            keras.layers.MaxPool3D(2, name="maxpool3d_2"),
            SeluConv3D(filters=128, kernel_size=3, name="selu_conv3d_3"),
            keras.layers.MaxPool3D(2, name="maxpool3d_3"),
            SeluConv3D(filters=256, kernel_size=3, name="selu_conv3d_4"),
            keras.layers.MaxPool3D(2, name="maxpool3d_4"),
            keras.layers.Flatten(name="flatten"),
            keras.layers.Dense(1, activation="sigmoid", name="final_dense"),
        ],
        name="3d_cnn",
    )
    return model


m = build_3d_cnn()
m.summary()

In [ ]:
def build_pretrained_3d_cnn(freezing=True):
    pretrained_3d_cnn = keras.models.load_model("models/lidc-num-nodules-3dcnn.h5")
    pretrained_3d_cnn.pop()  # remove last dense layer
    model = keras.Sequential(
        [
            keras.layers.InputLayer(SCAN_SHAPE, name="pretrained_input"),
            pretrained_3d_cnn,
            keras.layers.Dense(1, activation="sigmoid", name="final_dense"),
        ],
        name="pretrained_3d_cnn",
    )
    pretrained_3d_cnn.trainable = not freezing
    return model


m = build_pretrained_3d_cnn()
m.summary()

In [ ]:
test_perc = 0.1
val_perc = 0.1
learning_rate = 1e-5
batch_size = 8
patience = 15

In [ ]:
trainval_dataset, test_dataset = train_test_split(
    covid_dataset, test_perc=test_perc, cardinality=covid_samples
)
train_dataset, val_dataset = train_test_split(trainval_dataset, test_perc=val_perc)
test_dataset = test_dataset.batch(1)
val_dataset = (
    val_dataset.cache().batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
)
train_dataset = (
    train_dataset.cache()  # must be called before shuffle
    .shuffle(buffer_size=1024, reshuffle_each_iteration=True)
    .batch(batch_size)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [ ]:
cnn = build_3d_cnn()
# cnn = build_pretrained_3d_cnn()
cnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[
        keras.metrics.TruePositives(name="tp"),
        keras.metrics.FalsePositives(name="fp"),
        keras.metrics.TrueNegatives(name="tn"),
        keras.metrics.FalseNegatives(name="fn"),
        keras.metrics.Precision(name="precision"),
        keras.metrics.Recall(name="recall"),
        keras.metrics.AUC(name="auc"),
        keras.metrics.BinaryAccuracy(name="accuracy"),
    ],
)
history = cnn.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1000,
    verbose=1,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=patience,
            restore_best_weights=True,
        ),
    ],
)

In [ ]:
plot_loss_history(history)

In [ ]:
print(cnn.evaluate(test_dataset, return_dict=True))

In [ ]:
rounds = 20
fracs = (1, 2, 5, 7, 10)
test_perc = 0.1
val_perc = 0.1
learning_rate = 1e-5
batch_size = 8
patience = 10
num_epochs = 1000
metrics = [
    keras.metrics.AUC(name="auc", num_thresholds=1000),
]

In [ ]:
data = defaultdict(list)
wo_pt_histories = []
w_pt_histories = []
w_pt_freezing_histories = []
for r in tqdm(range(rounds)):
    print(f" {r = } ".center(50, "="))
    full_train_dataset, test_dataset = train_test_split(
        covid_dataset, test_perc=test_perc, cardinality=covid_samples
    )
    # print(f"Test size: {sum(1 for _ in test_dataset)}")
    test_dataset = test_dataset.batch(batch_size)
    full_train_dataset, val_dataset = train_test_split(
        full_train_dataset, test_perc=val_perc
    )
    # print(f"Val size: {sum(1 for _ in val_dataset)}")
    val_dataset = val_dataset.batch(batch_size)
    wo_pt_frac_histories = []
    w_pt_frac_histories = []
    w_pt_freezing_frac_histories = []
    for frac in fracs:
        print(f" {frac = } ".center(25, "="))
        train_dataset = (
            full_train_dataset.shuffle(1024, reshuffle_each_iteration=False)
            .shard(num_shards=frac, index=0)
            .cache()  # must be called before shuffle
            .shuffle(buffer_size=1024, reshuffle_each_iteration=True)
            .batch(batch_size)
            .prefetch(tf.data.experimental.AUTOTUNE)
        )
        # print(f"Train size: {sum(1 for _ in train_dataset.unbatch())}")

        cnn = build_3d_cnn()
        cnn.compile(
            optimizer=keras.optimizers.Adam(learning_rate),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=metrics,
        )
        history = cnn.fit(
            train_dataset,
            validation_data=val_dataset,
            epochs=num_epochs,
            verbose=0,
            callbacks=[
                keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    patience=patience,
                    restore_best_weights=True,
                )
            ],
        )
        wo_pt_frac_histories.append(history)
        test_metrics = cnn.evaluate(test_dataset, return_dict=True, verbose=False)
        wo_pt_auc = test_metrics["auc"]
        print(f"{wo_pt_auc = }")
        data["nn_type"].append("w/o pre-training")
        data["train_size_fraction"].append(frac)
        data["metric"].append("AUC")
        data["metric_value"].append(wo_pt_auc)

        cnn = build_pretrained_3d_cnn(freezing=False)
        cnn.compile(
            optimizer=keras.optimizers.Adam(learning_rate),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=metrics,
        )
        history = cnn.fit(
            train_dataset,
            validation_data=val_dataset,
            epochs=num_epochs,
            verbose=0,
            callbacks=[
                keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    patience=patience,
                    restore_best_weights=True,
                )
            ],
        )
        w_pt_frac_histories.append(history)
        test_metrics = cnn.evaluate(test_dataset, return_dict=True, verbose=False)
        w_pt_auc = test_metrics["auc"]
        print(f"{w_pt_auc = }")
        data["nn_type"].append("w/ pre-training (w/o freezing)")
        data["train_size_fraction"].append(frac)
        data["metric"].append("AUC")
        data["metric_value"].append(w_pt_auc)

    wo_pt_histories.append(wo_pt_frac_histories)
    w_pt_histories.append(w_pt_frac_histories)

In [ ]:
g = sns.catplot(
    x="train_size_fraction",
    y="metric_value",
    col="metric",
    hue="nn_type",
    kind="point",
    data=pd.DataFrame(data),
)
# for ax in g.axes[0]:
#    ax.axhline(color="r", ls="--")

In [ ]:
round_index = 0
frac_index = 1
plt.plot(
    wo_pt_histories[round_index][frac_index].history["loss"],
    "--",
    label="w/o pre-training - train loss",
)
plt.plot(
    wo_pt_histories[round_index][frac_index].history["val_loss"],
    label="w/o pre-training - val loss",
)
plt.plot(
    w_pt_histories[round_index][frac_index].history["loss"],
    "--",
    label="w/ pre-training, w/o conv freezing - train loss",
)
plt.plot(
    w_pt_histories[round_index][frac_index].history["val_loss"],
    label="w/ pre-training, w/o conv freezing - val loss",
)
# plt.plot(
#    w_pt_freezing_histories[round_index][frac_index].history["loss"],
#    "--",
#    label="w/ pre-training, w/ conv freezing - train loss",
# )
# plt.plot(
#    w_pt_freezing_histories[round_index][frac_index].history["val_loss"],
#    label="w/ pre-training, w/ conv freezing - val loss",
# )
plt.legend()

In [ ]:
data = defaultdict(list)
wo_pt_histories = []
w_pt_histories = []
for _ in tqdm(range(rounds)):
    trainval_dataset, test_dataset = train_test_split(
        covid_dataset, test_perc=test_perc, cardinality=covid_samples
    )
    test_dataset = test_dataset.batch(batch_size)
    full_train_dataset, val_dataset = train_test_split(
        trainval_dataset, test_perc=val_perc
    )
    val_dataset = val_dataset.batch(batch_size)
    wo_pt_frac_histories = []
    w_pt_frac_histories = []
    for frac in fracs:
        train_dataset = (
            full_train_dataset.shuffle(1024, reshuffle_each_iteration=False)
            .shard(num_shards=frac, index=0)
            .cache()  # must be called before shuffle
            .shuffle(buffer_size=1024, reshuffle_each_iteration=True)
            .batch(batch_size)
            .prefetch(tf.data.experimental.AUTOTUNE)
        )

        cnn = build_3d_cnn()
        cnn.compile(
            optimizer=keras.optimizers.Adam(learning_rate),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=metrics,
        )
        history = cnn.fit(
            train_dataset,
            validation_data=val_dataset,
            epochs=num_epochs,
            verbose=0,
            callbacks=[
                keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    patience=patience,
                    restore_best_weights=True,
                )
            ],
        )
        wo_pt_frac_histories.append(history)
        test_metrics = cnn.evaluate(test_dataset, return_dict=True, verbose=False)
        data["nn_type"].append("w/o pretraining")
        data["train_size_fraction"].append(frac)
        data["metric"].append("AUC")
        data["metric_value"].append(test_metrics["auc"])

        cnn = build_pretrained_3d_cnn(True)
        cnn.compile(
            optimizer=keras.optimizers.Adam(learning_rate),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=metrics,
        )
        history = cnn.fit(
            train_dataset,
            validation_data=val_dataset,
            epochs=num_epochs,
            verbose=0,
            callbacks=[
                keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    patience=patience,
                    restore_best_weights=True,
                )
            ],
        )
        w_pt_frac_histories.append(history)
        test_metrics = cnn.evaluate(test_dataset, return_dict=True, verbose=False)
        data["nn_type"].append("w/ pretraining (conv freezing)")
        data["train_size_fraction"].append(frac)
        data["metric"].append("AUC")
        data["metric_value"].append(test_metrics["auc"])
    wo_pt_histories.append(wo_pt_frac_histories)
    w_pt_histories.append(w_pt_frac_histories)

In [ ]:
sns.catplot(
    x="train_size_fraction",
    y="metric_value",
    hue="nn_type",
    col="metric",
    kind="point",
    data=pd.DataFrame(data),
)

In [ ]:
round_index = 1
frac_index = 0
plt.plot(
    wo_pt_histories[round_index][frac_index].history["loss"],
    "--",
    label="w/o pre-training - train loss",
)
plt.plot(
    wo_pt_histories[round_index][frac_index].history["val_loss"],
    label="w/o pre-training - val loss",
)
plt.plot(
    w_pt_histories[round_index][frac_index].history["loss"],
    "--",
    label="w/ pre-training, w/ conv freezing - train loss",
)
plt.plot(
    w_pt_histories[round_index][frac_index].history["val_loss"],
    label="w/ pre-training, w/ conv freezing - val loss",
)
plt.legend()

In [ ]:
k = 10
val_perc = 0.1
learning_rate = 1e-5
batch_size = 8
patience = 10
num_epochs = 1000

In [ ]:
wo_pt_histories = []
wo_pt_metrics = [
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
    keras.metrics.AUC(name="auc", num_thresholds=1000),
    keras.metrics.BinaryAccuracy(name="accuracy"),
]
w_pt_histories = []
w_pt_metrics = [
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
    keras.metrics.AUC(name="auc", num_thresholds=1000),
    keras.metrics.BinaryAccuracy(name="accuracy"),
]
for fold_id, (train_dataset, test_dataset) in tqdm(
    enumerate(kfolds(k, covid_dataset, cardinality=covid_samples)), total=k
):
    test_dataset = test_dataset.batch(1)
    train_dataset, val_dataset = train_test_split(train_dataset, test_perc=val_perc)
    val_dataset = val_dataset.batch(batch_size)
    train_dataset = (
        train_dataset.cache()  # must be called before shuffle
        .shuffle(buffer_size=1024, reshuffle_each_iteration=True)
        .batch(batch_size)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )

    cnn = build_3d_cnn()
    cnn.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.BinaryCrossentropy(),
    )
    history = cnn.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=num_epochs,
        verbose=1,
        callbacks=[
            keras.callbacks.EarlyStopping(
                monitor="val_loss",
                patience=patience,
                restore_best_weights=True,
            )
        ],
    )
    wo_pt_histories.append(history)
    for test_x, test_y in test_dataset.as_numpy_iterator():
        pred_y = cnn(test_x, training=False)
        for metric in wo_pt_metrics:
            metric.update_state(test_y, pred_y)

    cnn = build_pretrained_3d_cnn(True)
    cnn.compile(
        optimizer=keras.optimizers.Adam(1e-5),
        loss=keras.losses.BinaryCrossentropy(),
    )
    history = cnn.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=num_epochs,
        verbose=0,
        callbacks=[
            keras.callbacks.EarlyStopping(
                monitor="val_loss",
                patience=patience,
                restore_best_weights=True,
            )
        ],
    )
    w_pt_histories.append(history)
    for test_x, test_y in test_dataset.as_numpy_iterator():
        pred_y = cnn(test_x, training=False)
        for metric in w_pt_metrics:
            metric.update_state(test_y, pred_y)

print(" total ".center(50, "="))
print("Without pretraining: ")
for metric in wo_pt_metrics:
    print(f"{metric.name}: {metric.result()}")
print("")

print("With pretraining (w/ conv freezing): ")
for metric in w_pt_metrics:
    print(f"{metric.name}: {metric.result()}")

In [ ]:
index = 0
plt.plot(
    wo_pt_histories[index].history["loss"], "--", label="w/o pre-training - train loss"
)
plt.plot(
    wo_pt_histories[index].history["val_loss"], label="w/o pre-training - val loss"
)
plt.plot(
    w_pt_histories[index].history["loss"],
    "--",
    label="w/ pre-training, w/o conv freezing - train loss",
)
plt.plot(
    w_pt_histories[index].history["val_loss"],
    label="w/ pre-training, w/o conv freezing - val loss",
)
plt.plot(
    w_pt_conv_histories[index].history["loss"],
    "--",
    label="w/ pre-training, w/ conv freezing - train loss",
)
plt.plot(
    w_pt_conv_histories[index].history["val_loss"],
    label="w/ pre-training, w/ conv freezing - val loss",
)
plt.legend()